### Bibliotecas e outros

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import ast
from wordcloud import WordCloud
from sklearn.cluster import KMeans

CSV_FILE = "./relatorio_rt.csv"

In [ ]:
def read_data():
  df = pd.read_csv(CSV_FILE)

  # Filter out outliers for '%_cumprida': values must be between 0 and 100
  df = df[(df['%_cumprida'] >= 0) & (df['%_cumprida'] <= 100)]

  return df

### Perguntas norteadoras de como provar o valor da Mentoria

Pergunta: Os alunos com a maior porcentagem de metas alcançadas são os que apresentam as emoções mais positivas?

In [ ]:
def primeira_pergunta():
    df = read_data().copy()

    # Criar faixas de desempenho
    df["faixa_desempenho"] = pd.cut(
        df["%_cumprida"],
        bins=[-1, 33, 66, 100],
        labels=["Baixa", "Média", "Alta"]
    )

    sns.set_theme(style="whitegrid", palette="pastel")

    plt.figure(figsize=(8, 5))

    # Barplot da média do saldo emocional por faixa
    sns.barplot(
        data=df,
        x="faixa_desempenho",
        y="resultado_subtracao_emocoes",
        estimator="mean",
        errorbar=("ci", 95),
        palette="pastel"
    )

    plt.ylim(0, 5)

    plt.title("Média do Saldo Emocional por Faixa de Metas Cumpridas")
    plt.xlabel("Faixa de Metas Cumpridas")
    plt.ylabel("Saldo Emocional Médio")
    plt.tight_layout()
    plt.show()

primeira_pergunta()

Pergunta: Há mentores cujos alunos demonstram, em média, emoções mais positivas?

In [ ]:
def segunda_pergunta(top_n: int = 10):
    df = read_data().copy()

    mentor_stats = (
        df.groupby("nome_mentor")
        .agg(
            avg_positive_emotions=("qtd_emocoes_positivas", "mean"),
            total_sessions=("qtd_emocoes_positivas", "count"),
            unique_students=("nome_aluno_bubble", "nunique")
        )
        .reset_index()
    )

    mentor_stats = mentor_stats.sort_values("avg_positive_emotions", ascending=False)

    sns.set_theme(style="whitegrid", palette="pastel")

    plot_df = mentor_stats.head(top_n)

    plt.figure(figsize=(10, 6))
    sns.barplot(
        data=plot_df,
        x="avg_positive_emotions",
        y="nome_mentor",
        palette="pastel"
    )
    plt.title(f"Mentores com Maior Média de Emoções Positivas (Top {top_n})")
    plt.xlabel("Média de Emoções Positivas por Sessão")
    plt.ylabel("Mentor")
    plt.tight_layout()
    plt.show()
    return mentor_stats

# Execução
segunda_pergunta()

Pergunta: Como os alunos evoluíram ao longo do ano?

In [ ]:
def terceira_questao():
    df = read_data().copy()

    month_order = [
        "Janeiro", "Fevereiro", "Março", "Abril", "Maio", "Junho",
        "Julho", "Agosto", "Setembro", "Outubro", "Novembro", "Dezembro"
    ]
    df["mes_ordenado"] = pd.Categorical(df["mes"], categories=month_order, ordered=True)

    monthly_emotional = df.groupby("mes_ordenado", observed=False)["resultado_subtracao_emocoes"].mean().reset_index()
    monthly_goals = df.groupby("mes_ordenado", observed=False)["%_cumprida"].mean().reset_index()

    sns.set_theme(style="whitegrid")
    fig, axes = plt.subplots(1, 1, figsize=(12, 10), sharex=True)
    sns.lineplot(data=monthly_goals, x="mes_ordenado", y="%_cumprida",
                 marker="o", color="#AF7AC5", linewidth=2, ax=axes, label="Desempenho Mensal")

    media_geral_meta = monthly_goals["%_cumprida"].mean()
    axes.axhline(media_geral_meta, color='r', linestyle='--', alpha=0.5, label=f'Média Geral ({media_geral_meta:.1f}%)')

    axes.set_title("Sazonalidade de Desempenho: Meses abaixo da média", fontsize=14)
    axes.set_ylabel("% Metas Cumpridas")
    axes.set_xlabel("Mês")
    axes.legend()

    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

terceira_questao()

Pergunta: Há alguma relação entre a quantidade de aulas por mês e o desempenho médio dos alunos?

In [ ]:
def quarta_questao():
  df = read_data().copy()

  df["data_aula"] = pd.to_datetime(df["data_aula"], errors="coerce", dayfirst=True)

  df["mes_nome"] = df["data_aula"].dt.month_name()

  freq_df = (
    df.groupby(["nome_aluno_bubble", "mes_nome"])
    .agg(
      aulas_mensais=("data_aula", "count"),
      media_metas=('%_cumprida', "mean")
    )
    .reset_index()
  )

  sns.set_theme(style="whitegrid", palette="pastel")

  plt.figure(figsize=(8, 5))
  sns.lineplot(
    data=freq_df,
    x="aulas_mensais",
    y="media_metas",
    estimator="mean",
    ci=None,
    marker="o"
  )
  plt.title("Tendência entre Aulas Mensais e Desempenho Médio")
  plt.xlabel("Aulas Mensais por Aluno")
  plt.ylabel("Média de Metas Cumpridas (%)")
  plt.tight_layout()
  plt.show()

quarta_questao()

Pergunta: Houve maior engajamento por parte dos alunos, ao longo do ano?

In [ ]:
def quinta_questao():
  df['data_aula'] = pd.to_datetime(df['data_aula'], format='%d/%m/%Y', errors='coerce')

  df_clean = df.dropna(subset=['data_aula']).copy()

  df_clean['mes_ano'] = df_clean['data_aula'].dt.to_period('M')

  aulas_por_mes = df_clean['mes_ano'].value_counts().sort_index()

  aulas_por_mes.index = aulas_por_mes.index.astype(str)

  # gráfico de barras
  plt.figure(figsize=(15, 7))
  aulas_por_mes.plot(kind='bar')
  plt.title('Quantidade de Aulas por Mês', fontsize=16)
  plt.xlabel('Mês')
  plt.ylabel('Número de Aulas')
  plt.xticks(rotation=45, ha='right')
  plt.grid(axis='y')
  plt.tight_layout()
  plt.show()

quinta_questao()

Pergunta: Existe alguma relação entre a duração da aula e porcentagem de metas cumpridas?

In [ ]:
def sexta_pergunta():
    try:
        df_principal = read_data().copy()

        df_para_plotar = df_principal.copy()

        df_para_plotar['duracao_aula'] = pd.to_numeric(df_para_plotar['duracao_aula'])

        df_para_plotar['duracao_aula_minutos'] = df_para_plotar['duracao_aula'] * 60

        bins = [0, 50, 75, 100, 125, 150, 175, 200, 225, np.inf]
        labels = ['0-50', '51-75', '76-100', '101-125', '126-150', '151-175', '176-200', '201-225', '225+']

        df_para_plotar['duracao_aula_intervalo'] = pd.cut(df_para_plotar['duracao_aula_minutos'],
                                                        bins=bins,
                                                        labels=labels,
                                                        right=True)

        df_agg_detailed = df_para_plotar.groupby('duracao_aula_intervalo', observed=True).agg(
            perc_cumprida_media=('%_cumprida', 'mean'),
            emocoes_positivas_media=('qtd_emocoes_positivas', 'mean'),
            emocoes_negativas_media=('qtd_emocoes_negativas', 'mean')
        ).reset_index()

        fig, ax1 = plt.subplots(figsize=(14, 8))

        sns.lineplot(data=df_agg_detailed, x='duracao_aula_intervalo', y='perc_cumprida_media',
                    marker='o', ax=ax1, color='tab:blue', label='% Média Cumprida')
        ax1.set_xlabel('Intervalo de Duração da Aula (minutos)', fontsize=12)
        ax1.set_ylabel('% de Metas Cumpridas (Média)', color='tab:blue', fontsize=12)
        ax1.tick_params(axis='y', labelcolor='tab:blue')
        ax1.set_ylim(bottom=0)

        ax2 = ax1.twinx()

        sns.lineplot(data=df_agg_detailed, x='duracao_aula_intervalo', y='emocoes_positivas_media',
                    marker='s', ax=ax2, color='tab:green', label='Média de Emoções Positivas')

        sns.lineplot(data=df_agg_detailed, x='duracao_aula_intervalo', y='emocoes_negativas_media',
                    marker='x', ax=ax2, color='tab:red', label='Média de Emoções Negativas')

        ax2.set_ylabel('Média de Emoções Percebidas', color='black', fontsize=12)
        ax2.tick_params(axis='y', labelcolor='black')
        ax2.set_ylim(bottom=0)

        lines, labels_ax1 = ax1.get_legend_handles_labels()
        lines2, labels_ax2 = ax2.get_legend_handles_labels()
        ax2.legend(lines + lines2, labels_ax1 + labels_ax2, loc='upper left')

        plt.title('Metas Cumpridas, Emoções Positivas e Negativas por Duração da Aula', fontsize=16)
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.tight_layout()
        plt.show()

    except Exception as e:
        print(f"Ocorreu um erro")
        print(e)

sexta_pergunta()